## EyeON: Eye on Operational Networks
### a firmware inventory/threat analysis tool


In [ ]:
from eyeon import observe
from pprint import pprint

### Objects
EyeON consists of two parts - an `observe` call and a `parse` call. `observe` works on a single file to return a suite of identifying metrics. `parse` calls `observe` recursively, returning an observation for each file in a directory. Both of these can be run either from a library import or a `CLI` command.

In [ ]:
obs = observe.Observe("./tests/Obsidian.1.1.9.exe")
ols = observe.Observe("./tests/ls")

### Data Standard
Depending on the file type, e.g. PE or ELF, different observations will be collected. 
For instance, PE files typically contain more metadata and have signature information. Here we show some high-level characteristics, and we can dig into the certificates more thoroughly.

In [ ]:
print("authentihash:", obs.authentihash)
print("filename:", obs.filename)
print("file magic:", obs.magic)
print("certificates:", obs.signatures["valid"])

In [ ]:
for sig, vs in obs.signatures["signatures"].items():
    print("digest algorithm:", vs["digest_algorithm"])
    print("digest value:", sig)
    print("signers", vs["signers"])
    print("cert validation:", vs["verification"])
    for cert in vs["certs"]:
        pprint(cert)

There is also a Command Line component installed with the `eyeon` library containing 2 options: `eyeon observe` and `eyeon parse`.
`observe` generates output for a single file, whereas `parse` scans a directory.

It can be called as below (note `!` executes a terminal command):

In [ ]:
! eyeon --help

In [ ]:
! eyeon observe --output-dir ./outputs ./tests/Obsidian.1.1.9.exe
! jq . ./outputs/Obsidian.*

In [ ]:
G = nx.DiGraph(name="Signing Certs")
label_dict = {}
sample_cert = None


In [ ]:
#for root, dirs, files in os.walk(exe):
# for file in os.listdir(exe):
#    path = root.split(os.sep)
#    print((len(path) - 1) * '---', os.path.basename(root))
#   for file in files:
#    print(len(path) * '---', file)
file = "Obsidian.1.1.9.exe"
pe = lief.parse(f"tests/{file}")
# for sig in pe.signatures:
sig = pe.signatures[0]
for crt in sig.certificates:
# print(crt)
# Write cert (in DER format)
    with open(f"tests/outputs/{crt.subject}.crt", "wb") as binary_file:
      binary_file.write(crt.raw)
    # Add to graph
    try:
      G.add_nodes_from([crt.subject,crt.issuer])
      G.add_edge(crt.subject,crt.issuer)
      label_dict[crt.subject]=crt.subject
      G.add_nodes_from([crt.subject,crt.issuer])
      G.add_edge(crt.subject,crt.issuer)
    except nx.NetworkXError:
      print('Yikes!')
      print(crt.subject)
      print(crt.issuer)
    spl = str(crt.subject).split(', ')
    subj = {}
    for i in spl:
        j = i.split("=")
        try:
            subj[j[0]] = j[1]
        except:
            print(j)
            subj[j[0]] = ""
    print(crt.subject)
    print(subj)


In [ ]:
nx.draw_spring(G,with_labels=True, labels=label_dict, )


In [ ]:
# %pip install matplotlib

In [ ]:
G.in_degree

In [ ]:
spl = str(crt.subject).split(',')
subj = {}
for i in spl:
    j = i.split("=")
    subj[j[0]] = j[1]


In [ ]:
subj

In [ ]:
help(nx.draw_networkx)